In [2]:
import json

In [3]:
dir_8s = "../new_results/gpt45_new/germanic_new/8s"
dir_8t = "../new_results/gpt45_new/germanic_new/8t"

In [4]:
def load_jsonl(file):
    lines = open(file, 'r').readlines()
    data = [json.loads(line.strip()) for line in lines]
    return data

In [5]:
def print_topk_diffs(lang):

    hau_8s = load_jsonl(dir_8s + f"/{lang}/responses.json")
    hau_8t = load_jsonl(dir_8t + f"/{lang}/responses.json")
    
    for (eg, prev_eg) in zip(hau_8t, hau_8s):
        eg['diff'] = sum([int(lbl != prev_lbl) for (lbl, prev_lbl) in zip(eg['pred_labels'], prev_eg['pred_labels'])])
        eg['prev_labels'] = prev_eg['pred_labels']
        
    hau_8t_sorted = sorted(hau_8t, key=lambda x: -x['diff'])
    
    for eg in hau_8t_sorted[:10]:
        print('Toks: ' + eg['input'])
        print('8s: ' + ' '.join(eg['prev_labels']))
        print('8t: ' + ' '.join(eg['pred_labels']))
        print('Gd: ' + ' '.join([lbl.rsplit("_", 1)[1] for lbl in eg["output"].strip().split(" ")]))
        print('')

In [116]:
for lang in ['hau', 'ibo', 'kin', 'lug', 'luo']:
    print(lang)
    print_topk_diffs(lang)

hau


FileNotFoundError: [Errno 2] No such file or directory: '../new_results/gpt4_new/germanic/8s/hau/responses.json'

In [121]:
for lang in ['fo', 'got', 'gsw']:
    print(lang)
    print_topk_diffs(lang)

fo
Toks: Naksos er kent fyri síni eplir í restini av Grikkalandi .
8s: PROPN AUX VERB ADP DET NOUN ADP NOUN ADP PROPN PUNCT
8t: PROPN AUX ADJ ADP PRON NOUN ADP NOUN ADP PROPN PUNCT
Gd: PROPN VERB VERB ADP DET NOUN ADP NOUN ADP PROPN PUNCT

Toks: Hetta verður ofta nevnt at hava “ formansskapin í ES ” .
8s: PROPN AUX ADV VERB SCONJ VERB PUNCT NOUN ADP PROPN PUNCT PUNCT
8t: PRON AUX ADV VERB PART VERB PUNCT NOUN ADP PROPN PUNCT PUNCT
Gd: PRON VERB ADV VERB PART VERB PUNCT NOUN ADP NOUN PUNCT PUNCT

Toks: Meginparturin av fólkinum er katolikkar .
8s: NOUN ADP NOUN AUX NOUN PUNCT
8t: ADJ ADP NOUN AUX ADJ PUNCT
Gd: NOUN ADP NOUN VERB NOUN PUNCT

Toks: Hetta kom í lag við kongsbrævi , dagfest 19. mars .
8s: NOUN VERB ADP NOUN ADP NOUN PUNCT VERB NUM NOUN PUNCT
8t: PRON VERB ADP NOUN ADP NOUN PUNCT VERB ADJ NOUN PUNCT
Gd: PRON VERB ADP NOUN ADP NOUN PUNCT VERB ADJ NOUN PUNCT

Toks: Gáta : Ein gáta er ein stutt søga , sum endar við einum spurningi .
8s: NOUN PUNCT DET NOUN AUX DET ADJ NOUN PUNC

In [6]:
## confusion matrices for NER
ner_lbls = ['LOC', 'PER', 'ORG', 'DATE', 'O']
ner_lbls = {lbl : idx for lbl,idx in zip(ner_lbls,range(len(ner_lbls)))}

pos_lbls = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
pos_lbls = {lbl : idx for lbl,idx in zip(pos_lbls,range(len(pos_lbls)))}

def cmat(langpath, lbls):
    egs = load_jsonl('../new_results/gpt4_new/' + langpath + f"/responses.json")
    cmat = [[0 for _ in range(len(lbls))] for _ in range(len(lbls))]

    for eg in egs:
        gold_lbls = [lbl.rsplit("_", 1)[1].split('-')[-1] for lbl in eg["output"].strip().split(" ")]
        for (i, lbl) in enumerate(gold_lbls):
            gold_lbl_idx = lbls[lbl]
            pred_lbl_idx = lbls[eg['pred_labels'][i].split('-')[-1]]
            cmat[gold_lbl_idx][pred_lbl_idx] += 1
      
    return cmat

In [8]:
def print_cmat(cmat, lbls):
    lbl_keys = list(lbls.keys())
    print(".," + ",".join(lbl_keys))
    for (ridx, row) in enumerate(cmat):
        print(lbl_keys[ridx], end=",")
        for cell in row:
            print(cell, end=",")
        print()

In [ ]:
def compress_cmat(cmat):
    o_col = [0 for _ in range(len(cmat))]
    o_row = [0 for _ in range(len(cmat))]
    
    n_rdx = 0
    for i in range(len(cmat)):
        if cmat[i][i] < 100:
            n_rdx += 1
            for j in range(len(cmat[i])):
                o_col[i] += cmat[i][j]
            for j in range(len(cmat[i])):
                o_row[i] += cmat[j][i]
                

In [74]:
for lang in ['hau', 'ibo', 'kin', 'lug', 'luo']:
    print(lang)
    print_cmat(cmat(f'african/8t/{lang}', ner_lbls), ner_lbls)
    print()

hau
.,LOC,PER,ORG,DATE,O
LOC,71,0,8,0,1,
PER,0,85,0,0,2,
ORG,3,0,24,0,24,
DATE,1,1,0,20,69,
O,10,7,26,1,2393,

ibo
.,LOC,PER,ORG,DATE,O
LOC,60,3,1,0,2,
PER,0,86,0,0,0,
ORG,1,0,79,0,7,
DATE,0,0,0,29,33,
O,5,9,14,0,1542,

kin
.,LOC,PER,ORG,DATE,O
LOC,66,0,8,0,12,
PER,0,62,0,0,0,
ORG,3,0,59,0,4,
DATE,1,0,0,35,30,
O,4,10,15,3,1906,

lug
.,LOC,PER,ORG,DATE,O
LOC,53,1,4,0,8,
PER,0,120,0,0,8,
ORG,9,0,53,0,11,
DATE,0,0,0,12,35,
O,6,25,16,5,2033,

luo
.,LOC,PER,ORG,DATE,O
LOC,75,0,3,0,51,
PER,0,117,0,0,2,
ORG,24,0,27,0,52,
DATE,0,0,0,5,64,
O,2,8,14,4,2184,



In [11]:
for lang in ['fo', 'got', 'gsw']:
    print(lang)
    print_cmat(cmat(f'germanic_new/8t_desc/{lang}', pos_lbls), pos_lbls)
    print()

fo


FileNotFoundError: [Errno 2] No such file or directory: '../new_results/gpt4_new/germanic_new/8t_desc/fo/responses.json'

In [25]:
cmat_ner('african/8s/hau')

[[133, 1, 13, 2, 4],
 [0, 95, 0, 0, 4],
 [4, 0, 40, 0, 9],
 [0, 1, 0, 29, 104],
 [6, 14, 17, 0, 2421]]

In [40]:
def load_egs(langpath, eg_prefix):
    egs = load_jsonl('../new_results/gpt4_new/' + langpath + f"/responses.json")
    
    tgt_eg = None
    for eg in egs:
        if eg['input'].startswith(eg_prefix):
            tgt_eg = eg
            break

    if not tgt_eg:
        print('Could not find example')
        return

    print(f"input: {tgt_eg['input']}\n")
    print(f"Examples: [wrong->correct] \n")
    
    exemplars = tgt_eg['examples']
    for e in exemplars:
        a, b = list(zip(*[a.rsplit('_') for a in e.split(' ')]))
        a_str = ' '.join(a)
        for i, eg in enumerate(egs):
            if eg['input'] == a_str:
                print(i, end=" ")
                print(a_str, end="  \n")
                lbls = []
                acc = 0
                for (ptok, gtok) in zip(b, eg['gold_labels']):
                    if ptok == gtok:
                        lbls.append(ptok)
                        acc += 1
                    else:
                        lbls.append(f'[{ptok}->{gtok}]')
                print(f"{100*acc/len(a):.2f} {' '.join(lbls)}", end="  \n\n")
                break


In [44]:
load_egs('germanic_new/8t/gsw', "I cha der ihri")

input: I cha der ihri Telefonnummere gä , de nimmsch mou unverbindlech Kontakt uuf .

Examples: [wrong->correct] 

59 I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .  
83.33 PRON AUX [PART->ADP] PROPN VERB PUNCT PRON AUX PRON ADV ADJ CCONJ PRON [VERB->AUX] ADV [ADV->PART] VERB PUNCT  

4 De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .  
93.33 DET NOUN AUX DET NOUN VERB PUNCT DET NOUN AUX DET [PROPN->NOUN] VERB AUX PUNCT  

7 Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .  
77.27 DET NOUN AUX ADV [VERB->AUX] PUNCT SCONJ PRON ADV [VERB->ADV] ADJ [ADP->AUX] [VERB->AUX] PUNCT SCONJ PRON ADV AUX AUX [VERB->PART] VERB PUNCT  

70 Isch das e Sach gsi , bis mer se gfunge hei gha .  
92.31 AUX PRON DET NOUN AUX PUNCT SCONJ PRON PRON VERB AUX [VERB->AUX] PUNCT  

61 De Dialäkt muess zu de Gschecht und zum Inhaut vonere Werbig passe .  
100.00 DET NOUN AUX ADP DET NOUN CCON